In [14]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder \
    .master("spark://Nguyen-Duc-Trung.:7077") \
    .appName('Spark') \
    .getOrCreate()

23/06/04 13:12:46 WARN Utils: Your hostname, Nguyen-Duc-Trung resolves to a loopback address: 127.0.1.1; using 192.168.130.52 instead (on interface eth0)
23/06/04 13:12:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/04 13:12:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_green = spark.read.parquet('../data/green/2020/*.parquet')

In [5]:
df_green.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: integer (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [6]:
df_green.show(5)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2019-12-18 22:52:30|  2019-12-18 22:54:39|                 N|       1.0|         264|         264|            5.0|          0.0|        3.5|  0.5|    0.

In [7]:
df_yellow = spark.read.parquet('../data/yellow/2020/*.parquet')
df_yellow.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: integer (nullable = true)



In [8]:
#Change the column names : pickup_datetime, dropoff_datetime for the same columns in both dataframes 
df_green = df_green \
        .withColumnRenamed("lpep_pickup_datetime", "pickup_datetime") \
        .withColumnRenamed("lpep_dropoff_datetime", "dropoff_datetime")

df_yellow = df_yellow \
        .withColumnRenamed("tpep_pickup_datetime", "pickup_datetime") \
        .withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime")

In [10]:
len(df_green.columns), len(df_yellow.columns)

(20, 19)

In [9]:
#find which columns are different
set(df_green.columns) - set(df_yellow.columns)

{'ehail_fee', 'trip_type'}

In [11]:
set(df_yellow.columns) - set(df_green.columns)

{'airport_fee'}

In [12]:
#Check the common columns between the two dataframes  
set(df_green.columns).intersection(set(df_yellow.columns))

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'dropoff_datetime',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'pickup_datetime',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

In [13]:
#create a list of common columns
common_columns = list(set(df_green.columns).intersection(set(df_yellow.columns)))

In [15]:
#Add a new column to each dataframe to identify the company (yellow or green) : name column as service type
df_green_sel = df_green \
        .select(common_columns) \
        .withColumn('service_type', F.lit('green'))

df_yellow_sel = df_yellow \
        .select(common_columns) \
        .withColumn('service_type', F.lit('yellow'))

In [16]:
#Combine the two dataframes into one
df_trips = df_green_sel.unionAll(df_yellow_sel)

In [17]:
#Count the numbers of rows for each service type
df_trips.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 1734176|
|      yellow|24649092|
+------------+--------+



In [18]:
#Create temporary view for the dataframe df_trips in order to use SQL queries
df_trips.createOrReplaceTempView("trips_data")

In [19]:
spark.sql("""
SELECT 
    service_type,
    count(*) as count 
FROM 
    trips_data 
GROUP BY 
    service_type""") \
.show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 1734176|
|      yellow|24649092|
+------------+--------+



In [20]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone, -- Rename PULocationID column to revenue_zone
    date_trunc('month', pickup_datetime) AS revenue_month, -- Truncate pickup_datetime to month level
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare, -- Calculate monthly fare revenue
    SUM(extra) AS revenue_monthly_extra, -- Calculate monthly extra revenue
    SUM(mta_tax) AS revenue_monthly_mta_tax, -- Calculate monthly MTA tax revenue
    SUM(tip_amount) AS revenue_monthly_tip_amount, -- Calculate monthly tip revenue
    SUM(tolls_amount) AS revenue_monthly_tolls_amount, -- Calculate monthly tolls revenue
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge, -- Calculate monthly improvement surcharge revenue
    SUM(total_amount) AS revenue_monthly_total_amount, -- Calculate monthly total revenue
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge, -- Calculate monthly congestion surcharge revenue

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count, -- Calculate average monthly passenger count
    AVG(trip_distance) AS avg_montly_trip_distance -- Calculate average monthly trip distance
FROM
    trips_data -- Select data from trips_data
GROUP BY
    1, 2, 3 -- Group results by revenue_zone, revenue_month, and service_type
""")

In [ ]:
##Check the type of df_result

In [21]:
df_result.show()

+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|revenue_zone|      revenue_month|service_type|revenue_monthly_fare|revenue_monthly_extra|revenue_monthly_mta_tax|revenue_monthly_tip_amount|revenue_monthly_tolls_amount|revenue_monthly_improvement_surcharge|revenue_monthly_total_amount|revenue_monthly_congestion_surcharge|avg_montly_passenger_count|avg_montly_trip_distance|
+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|         120|2020-

In [22]:
#Write the dataframe to a parquet file 
df_result.coalesce(1).write.parquet('../data/report/revenue-2020', mode='overwrite')
